In [0]:
!pip install preprocessor
!pip install tweepy
!pip install contractions

  Stored in directory: /root/.cache/pip/wheels/98/c1/a2/21fbcfd80d76576bbf148991a66f00730f541f265c7600000f
Successfully built preprocessor


In [0]:
import os


import re
import nltk
import pickle
import contractions
from nltk import pos_tag
import numpy as np
import pandas as pd 
from pprint import pprint as pprint
import tweepy
import preprocessor as p
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem  import WordNetLemmatizer
from string import digits
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from google.colab import files
files.upload()

Saving pre-processed_data.csv to pre-processed_data.csv


{'pre-processed_data.csv': b',handle,catagory,tweet\n0,thedailyswarm,Music," Thank you.DIIV\'s Zachary Cole Talks Arrest, Addiction Watch: 10-Year-Old Adorably Shreds Slayer Listen to Karen O\'s First Song Off Her Solo LP The Other, Tolerable, Side of Ibiza\'s Music Scene Could This Be the Soundsystem of the Future? Nas Documents His Life in 20 Songs We\'re Surprised It Took Someone This Long to Turn Nicki Minaj Into a Cartoon Nas Documents His Life in 20 Songs We\'re Surprised It Took Someone This Long to Turn Nicki Minaj Into a Cartoon Pussy Riot Members Take the Kremlin to Court Childish Gambino Calls Out Peers In Own \'Control\'-Esque Offensive If Anyone Besides Morgan Freeman Narrated a Juggalo Doc, We\'d Be Extremely Disappointed Bobby Shmurda Isn\'t Good for Hip-Hop FKA Twigs: From Backup to Main Attraction Flying Lotus Announces Next Album, \'You\'re Dead!\' Kendrick Lamar Has 40 Songs Ready for Next Album; Dr. Dre\'s In There, Too 700 Indies Band Together to Launch \'Fair Digi

In [0]:
import pandas as pd
data = pd.read_csv('pre-processed_data.csv')
data.head()

,Unnamed: 0,handle,catagory,tweet
0,0,thedailyswarm,Music,"Thank you.DIIV's Zachary Cole Talks Arrest, A..."
1,1,soundofthecity,Music,Please check out our friends HIRAETH and drop ...
2,2,PasteMagazine,Music,MomoCon 2019: Anime &amp; Games Take Over Down...
3,3,SPINmagazine,Music,NaN
4,4,RollingStone,Music,The Season 2 finale sets up a wannabe cliffha...


In [0]:
data['catagory'].unique()

array(['Music', 'Sports', 'Food', 'Travel', 'Books', 'Tech'], dtype=object)

In [0]:
file = data['handle']

In [0]:
API_KEYS = 'hTXtPJWS4QamIazdzHLMZC9My'
API_SECRET_KEY = 'p1v4UtQQZZ1Afycxx1CF6oCqxsVDVrrQsMqcMa3Xjam57QAhx0'
ACCESS_TOKEN = '856419890-WUtORPcRsoC5r9g6CdUfTGORchDbF3rDiQAeYsW8'
ACCESS_TOKEN_SECRET = '8WbVRYHqzBKydyL7jCySE76E1wMz9z8j8WiCmH6oHOBx6'

In [0]:
auth = tweepy.OAuthHandler(API_KEYS, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [0]:
api = tweepy.API(auth)

In [0]:
stuff = api.user_timeline(screen_name='observer', count=100, include_rts=False)

In [0]:
stopword_tw = ['get', 'got', 'hey', 'hmm', 'hoo', 'let', 'ooo', 'par',
    'yer', 'didn', 'one', 'com', 'new', 'like', 'great',
    'make', 'top', 'awesome', 'best', 'good', 'wow', 'yes',
    'say', 'yay', 'would', 'thanks', 'thank', 'use',
    'should', 'could','best','really','see','want','nice',
    'while','know', 'trump', 'nyfw', 'iphone', 'hurricane',
    'rt', 'per', 'espn', 'soundcloud', 'ten', 'count', 
    'advance', 'newsletter','thedish', 'nycwff', 'thefword',
    'irma', 'fave', 'beer', 'stefan', 'aiga',
    'aigatogether', 'aigadesignconf', 'aigadg', 'aigagala',
    'ddc', 'tbt', 'whitneybiennial', 'calder', 'wknd',
    'pipilottirist', 'live', 'watch', 'check', 'video',
    'clip', 'today', 'tonight', 'week', 'year', 'month',
    'time', 'last', 'night', 'morning', 'yesterday',
    'tomorrow', 'day', 'first', 'love', 'nyc', 'city',
    'york', 'new', 'happy', 'need', 'look', 'back', 'right',
    'win', 'chance', 'enter', 'ever','pst','wha','yep', 'via',
    'app', 'twitter', 'streaming', 'stream', 'ask', 'amp',
    'beautiful', 'best', 'amazing', 'good', 'perfect', 'cute',
    'simple', 'love']

stop_list = set(stopwords.words('english') + stopword_tw)

In [0]:

def twitter_preprocess(messages):
#   print('here')
  new_tweet = []
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  
  for tweet in messages:
      tweet = re.sub(r"^https://t.co/[A-Za-z0-9]*\s"," ",tweet)
      tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", " ", tweet)
      tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", " ", tweet)
                                           # to substitute url's with a whitespace at the beginning, in between and at the end of a sentence
      
      tweet = re.sub("\.\.+"," ",tweet)    # to substitute more than one '.' between words with a space
      tweet = re.sub("-$","",tweet)        # to remove '-' at the end of a sentence
      tweet = re.sub(r"^ +","",tweet)      # to remove one or more whitespace at the beginning of a sentence
      tweet = re.sub(r"  +"," ",tweet)     # to substitute one or more whitespace with a single space
      tweet = emoji_pattern.sub(r'', tweet)
      tweet = re.sub(r"(?:#|\@|https?\:\/\/|www\.)\S+","",tweet)
      
#       tweet = ''.join([word for word in tweet if word not in stop_list])
      new_tweet.append(tweet)
  return new_tweet

In [0]:
def get_tweet(screen_name, count=200, include_rts=False):
  try:
    pprint('getting tweet from {}'.format(screen_name))

    stuff = api.user_timeline(screen_name = screen_name, count=count, include_rts=include_rts)
    total_tweet = []

    for tweet in stuff:
      total_tweet.append(tweet.text)

  #   print(len(total_tweet))

    return ''.join(twitter_preprocess(total_tweet))
  
  except:
    return np.nan

In [0]:
def get_tweet_by_handle(x):
  return get_tweet(screen_name=x)

In [0]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('brown')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz

--2019-06-03 04:44:56--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  28.3MB/s    in 45s     

2019-06-03 04:45:42 (28.0 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [0]:
!gunzip -k cc.en.300.vec.gz

In [0]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('cc.en.300.vec', binary=False)
model.save_word2vec_format('wiki.el.txt', binary=False)

In [0]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [0]:

lemmatizer = WordNetLemmatizer()
stop_word = list(set(stopwords.words('english')))
stop_word = list(filter(lambda x : x!=str('not'),stop_word))

def noun_phrase(token):
  '''To extract important grammatical token from text'''

  key = pos_tag(token)
  data=[]
  for tag in key:
    if tag[1] == 'NN':
      data.append(tag[0])
  
  if len(data) == 0:
    return token
  
  return list(data)


def text_processing(text):
    
    #removing everything except letter and space
    temp = re.sub(r"[^A-Za-z ]+",r"",text)
    #lower case
    temp = temp.lower()
    #remove unnecessary spaces
    temp = " ".join(temp.split())
    #word tokenize
    temp = word_tokenize(temp)
    # remove stop words
    temp = [ item for item in temp if item not in stop_word]
    #lemmatize
    temp = [lemmatizer.lemmatize(item) for item in temp]
    
    return temp



def find_token(text):
    '''
    This function convert text, i.e string, into token
    after doing all required pre-processing

    argument: text -> input string to find token
    returns the token

    input: text of type string
    output: strings in list
    
    '''
    preprocess_text = text_processing(text)
    noun_phrase_token = noun_phrase(preprocess_text)
    
    return noun_phrase_token



In [0]:
def find_class(text):
    ''''
    This function return the classes on which the WFH reason
    belongs to 

    argument: text -> wfh reason 
    returns the classes on which wfh reason belongs

    input: text of type string
    output: text of type string
    '''

    assert isinstance(text,str)
    tokens = find_token(text)

    if len(tokens) == 0:
        return 'empty'
    
    list_group = ['music', 'sport', 'food', 'travel', 'book', 'technology']
    similar = []

    for compare in list_group:
        compare_with_all = []
        
        for token in tokens:
            try:
                similarity = model.similarity(compare,token)
                compare_with_all.append(similarity)
            except:
                compare_with_all.append(-1)
        
        similar.append(sum(compare_with_all))

    if max(similar) > 0:
        max_index = similar.index(max(similar))
        return list_group[max_index]
    
    return 'other'


In [0]:
file = list(file)

In [0]:
for handle in file:
  post = get_tweet_by_handle(handle)
  class_belong = find_class(post)
  filename = class_belong + '.txt'
  with open(filename, "a+") as myfile:
    myfile.write(handle+',')

'getting tweet from thedailyswarm'


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'getting tweet from soundofthecity'
'getting tweet from PasteMagazine'
'getting tweet from SPINmagazine'
'getting tweet from RollingStone'
'getting tweet from HipHopDX'
'getting tweet from BLABBERMOUTHNET'
'getting tweet from billboard'
'getting tweet from stereogum'
'getting tweet from musicalredhead'
'getting tweet from Zernerlaw'
'getting tweet from cspanmusiclaw'
'getting tweet from CopyrightLaw'
'getting tweet from gfiremark'
'getting tweet from dealfatigue'
'getting tweet from nymusicLawyer'
'getting tweet from TameraBennett'
'getting tweet from SteveGordonLaw'
'getting tweet from NoisyMusic'
'getting tweet from SoundCloud'
'getting tweet from Pitchfork'
'getting tweet from Ticketmaster'
'getting tweet from SPIN '
'getting tweet from NPRmusic'
'getting tweet from NME'
'getting tweet from nytimesmusic'
'getting tweet from Shazam'
'getting tweet from warnermusic'
'getting tweet from MTVMusic'
'getting tweet from VVoiceMusic'
'getting tweet from Bandsintown'
'getting tweet from song

'tech'

In [0]:
!pip install wget
import wget
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz'
wget.download(url)

import gzip
f = gzip.open(wget.download(url), 'rb')
# file_content = f.read()
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format(f.read(), binary=False)
model.save_word2vec_format('wiki.el.txt', binary=False)
f.close()

In [0]:
import gzip
f = gzip.open(filename, 'rb')
file_content = f.read()
f.close()

In [0]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format(file_content, binary=False)
model.save_word2vec_format('wiki.el.txt', binary=False)

'cc.en.300.vec.gz'

In [0]:
file_content